In [1]:
import os
import numpy as np

import ncnn

import torch
from img2mol.inference import Img2MolInference

/home/jh-li/mambaforge/envs/img2mol-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Local CDDD installation has not been found.


RDKit WARNING: [16:07:08] Enabling RDKit 2019.09.3 jupyter extensions
[16:07:08] Enabling RDKit 2019.09.3 jupyter extensions


In [2]:
# load model
model_ckpt = os.path.join("..", "img2mol_convert", "model", "model.ckpt")
device = "cuda:0" if torch.cuda.is_available() else "cpu"

img2mol = Img2MolInference(
    model_ckpt=model_ckpt,
    device=device,
)

Initializing Img2Mol Model with random weights.
Loading checkpoint: ../img2mol_convert/model/model.ckpt
Setting to `self.eval()`-mode.
Sending model to `cpu` device.
Succesfully created Img2Mol Inference class.


In [3]:
# read image
img_file = os.path.join("..", "examples", "QOiFla.png")
imgs = img2mol.read_image_to_tensor(img_file)

/home/jh-li/mambaforge/envs/img2mol-env/lib/python3.8/site-packages/torchvision/transforms/transforms.py:1297: UserWarning: The parameter 'resample' is deprecated since 0.12 and will be removed 0.14. Please use 'interpolation' instead.
  warnings.warn(
/home/jh-li/mambaforge/envs/img2mol-env/lib/python3.8/site-packages/torchvision/transforms/transforms.py:1427: UserWarning: The parameter 'resample' is deprecated since 0.12 and will be removed in 0.14. Please use 'interpolation' instead.
  warnings.warn(
/home/jh-li/mambaforge/envs/img2mol-env/lib/python3.8/site-packages/torchvision/transforms/transforms.py:1442: UserWarning: The parameter 'fillcolor' is deprecated since 0.12 and will be removed in 0.14. Please use 'fill' instead.
  warnings.warn(
/home/jh-li/mambaforge/envs/img2mol-env/lib/python3.8/site-packages/torchvision/transforms/transforms.py:332: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode 

In [4]:
# infer
with torch.no_grad():
    img2mol_out = img2mol.model(imgs)

[W NNPACK.cpp:51] Could not initialize NNPACK! Reason: Unsupported hardware.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. 

In [5]:
def test_inference(imgs: torch.Tensor):
    # torch.manual_seed(0)
    # in0 = torch.rand(1, 1, 224, 224, dtype=torch.float)
    out = []

    with ncnn.Net() as net:
        net.opt.use_fp16_packed = False
        net.opt.use_fp16_storage = False
        net.opt.use_fp16_arithmetic = False

        net.load_param(os.path.join("model", "img2mol.ncnn.param"))
        net.load_model(os.path.join("model", "img2mol.ncnn.bin"))

        for img in imgs:
            with net.create_extractor() as ex:
                ex.input("in0", ncnn.Mat(img.numpy()).clone())

                _, out0 = ex.extract("out0")
                out.append(np.array(out0))

    return torch.from_numpy(np.array(out))

In [6]:
ncnn_out = test_inference(imgs)

In [7]:
torch.cosine_similarity(img2mol_out, ncnn_out)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 0.9999, 1.0000, 1.0000, 1.0000, 0.9999,
        0.9999, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.9999, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000])

In [8]:
np.save("cddd_emb.npy", np.median(ncnn_out.clone().detach().numpy(), axis=0))